In [23]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np

In [24]:
bikes = pd.read_excel("../data/raw/dados-bsb-bike-agosto.xlsx")

In [25]:
bikes.head()

,Gênero do usuário,data nascimento,Cidade / Região Administrativa onde habita,Idade,Data,Horário de retirada,Horário de devolução,tempo de viagem,Estação de origem,% de retiradas/estação,Estação de Destino,% de devoluções/estação
0,M,29/03/1980 00:00:00,NaN,38,2018-08-01,05:34:25,05:42:49,00:08:24,11 - Rodoviária 2,0.044207,28 - CNMP - Conselho Nacional do Ministério Pú...,0.008516
1,M,1992-06-07 00:00:00,NaN,26,2018-08-01,05:39:51,05:44:13,00:04:22,6 - Rodoviária,0.047566,10 - Ministério dos Transportes,0.015021
2,M,17/02/1975 00:00:00,NaN,43,2018-08-01,05:48:09,05:56:35,00:08:26,11 - Rodoviária 2,0.044207,23 - STF,0.016733
3,M,1994-08-07 00:00:00,NaN,23,2018-08-01,05:49:58,06:12:44,00:22:46,11 - Rodoviária 2,0.044207,8 - Ministério da Defesa,0.011319
4,F,29/08/1973 00:00:00,NaN,44,2018-08-01,05:57:37,06:16:22,00:18:45,6 - Rodoviária,0.047566,43 - Biblioteca Central,0.018444


In [26]:
#define 6 intervalos entre 0 e 90 anos
buckets = np.linspace(0, 90, 6)

In [27]:
bikes['faixa_etaria'] = pd.cut(bikes['Idade'], buckets)

In [28]:
bikes.dtypes

Gênero do usuário                                     object
data nascimento                                       object
Cidade / Região Administrativa onde habita            object
Idade                                                  int64
Data                                          datetime64[ns]
Horário de retirada                                   object
Horário de devolução                                  object
tempo de viagem                                       object
Estação de origem                                     object
% de retiradas/estação                               float64
Estação de Destino                                    object
% de devoluções/estação                              float64
faixa_etaria                                        category
dtype: object

In [29]:
def transform_into_seconds(tempo_viagem: str) -> float:
    """
    Função que será utilizada para converter um tempo total de viagem (string) em segundos.
    """
    #transforma em datetime
    try:
        step_1 = datetime.datetime.strptime(tempo_viagem, '%H:%M:%S').time()
    except ValueError:
        step_1 = datetime.datetime.strptime("00:00:00", '%H:%M:%S').time()
    #converte em timedelta
    step_2 = timedelta(minutes=step_1.minute, seconds=step_1.second)
    return step_2.total_seconds()

In [30]:
bikes['tempo de viagem'] = bikes['tempo de viagem'].astype(str)

In [31]:
bikes['tempo de viagem'] = bikes['tempo de viagem'].apply(lambda x: transform_into_seconds(x))

In [32]:
# COMO NÃO HÁ DATA DE ENTREGA, NÃO É POSSÍVEL INFERIR A QUE DIA FOI DEVOLVIDO OS REGISTROS ABAIXO
# OS DADOS NÃO ESTÃO COMO UMA TIMESTAMP
bikes[bikes['tempo de viagem'] == 0].head()

,Gênero do usuário,data nascimento,Cidade / Região Administrativa onde habita,Idade,Data,Horário de retirada,Horário de devolução,tempo de viagem,Estação de origem,% de retiradas/estação,Estação de Destino,% de devoluções/estação,faixa_etaria
1159,M,20/09/1996 00:00:00,NaN,21,2018-08-01,17:40:17,11:41:59,0.0,5 - Setor Hoteleiro Norte,0.015320,11 - Rodoviária 2,0.052573,"(18.0, 36.0]"
1289,F,29/09/1965 00:00:00,DF,52,2018-08-01,18:23:51,10:24:00,0.0,46 - EQN 408/409,0.039713,42 - PAT,0.031240,"(36.0, 54.0]"
1442,F,30/08/1996 00:00:00,NaN,21,2018-08-01,19:40:41,14:02:52,0.0,46 - EQN 408/409,0.039713,35 - CLN 403,0.023366,"(18.0, 36.0]"
1540,M,1996-12-09 00:00:00,NaN,21,2018-08-01,22:52:45,07:54:23,0.0,46 - EQN 408/409,0.039713,43 - Biblioteca Central,0.018444,"(18.0, 36.0]"
1542,M,1997-10-09 00:00:00,DF,20,2018-08-01,22:59:10,07:55:04,0.0,42 - PAT,0.026105,43 - Biblioteca Central,0.018444,"(18.0, 36.0]"


In [33]:
#Existem 58 registros de Gênero O, como não existe metadado disponível para contextualizar, será excluído do conjunto de dados.
bikes[bikes['Gênero do usuário'] == 'O'].shape

(58, 13)

In [34]:
bikes = bikes[bikes['Gênero do usuário'] != 'O']

In [35]:
bikes[bikes['tempo de viagem'] == 0].shape

(355, 13)

In [36]:
#seleciona os registros que serão descartados
data_error = bikes[bikes['tempo de viagem'] == 0]

In [37]:
#seleciona os indices a serem eliminados
lista_indice_errors = data_error.index.tolist()

In [38]:
#retira do dataset
bikes.drop(lista_indice_errors, inplace=True)

In [39]:
#exporta arquivo com todo o conteúdo
bikes.to_csv("../data/interm/bikes_df.csv", sep=',', encoding='latin-1', index=False)

In [40]:
genero_usuarios_bikes = bikes.groupby('Gênero do usuário')['tempo de viagem'].count()

In [41]:
genero_usuarios_bikes = pd.DataFrame(genero_usuarios_bikes)

In [42]:
genero_usuarios_bikes.columns = ['numero_usuarios']
genero_usuarios_bikes.index = ['Feminino', 'Masculino']

In [43]:
genero_usuarios_bikes['percentual'] = genero_usuarios_bikes['numero_usuarios'].apply(lambda x : round(x/genero_usuarios_bikes['numero_usuarios'].sum(),2))

In [44]:
genero_usuarios_bikes.head()

,numero_usuarios,percentual
Feminino,12214,0.26
Masculino,34107,0.74


In [45]:
#exporta arquivo com dados de genero
genero_usuarios_bikes.to_csv("../data/interm/genero_usuarios_df.csv", sep=',')

In [46]:
#cria dataset para faixas etarias
faixa_etaria_usuarios_bike_df = bikes.groupby('faixa_etaria')['Gênero do usuário'].count()

In [47]:
faixa_etaria_usuarios_bike_df  = pd.DataFrame(faixa_etaria_usuarios_bike_df)

In [48]:
faixa_etaria_usuarios_bike_df.columns=['numero_usuarios']

In [50]:
#exporta arquivo com dados de faixas etárias
faixa_etaria_usuarios_bike_df.to_csv("../data/interm/faixa_etaria_usuarios_df.csv", sep=',')

# Analise das Estações mais utilizadas

In [51]:
bikes.groupby('Estação de origem')["Estação de Destino"].count()

Estação de origem
1 - Memorial JK                                         786
10 - Ministério dos Transportes                         620
11 - Rodoviária 2                                      2035
12 - Rodoviária 3                                      1677
13 - Liberty Mall                                       496
14 - Brasília Shopping                                  762
15 - Brasil 21                                         2190
16 - SRTVS                                              874
17 - Shopping Pátio Brasil                              925
18 - Praça do Povo                                      436
19 - Banco Central                                      584
2 - Praça Buriti                                        630
20 - SAS                                                400
21 - EQS 202 / 203                                      564
22 - 102 Sul                                            710
23 - STF                                                665
24 - Galeria          

In [52]:
#sanitiza o nome das estações
bikes['Estação de origem'] = bikes['Estação de origem'].apply(lambda x : x[x.index('-')+1:].strip())
bikes['Estação de Destino'] = bikes['Estação de Destino'].apply(lambda x : x[x.index('-')+1:].strip())

In [53]:
dados_fluxo_viagens = bikes.groupby(["Estação de origem", "Estação de Destino"])['Gênero do usuário'].count()

In [54]:
dados_fluxo_viagens = pd.DataFrame(dados_fluxo_viagens)

In [55]:
dados_fluxo_viagens.columns=['contagem_viagens']

In [56]:
dados_fluxo_viagens.head()

contagem_viagens
Estação de origem Estação de Destino                  
102 Sul           102 Sul                           96
                  Banco Central                      8
                  Brasil 21                         40
                  Brasília Shopping                  2
                  CLN 204                            5

In [57]:
dados_fluxo_viagens.shape

(1902, 1)

In [58]:
dados_fluxo_viagens.to_csv("../data/interm/fluxo_trajetos_contagem_completo.csv", sep=',', encoding='latin-1')

In [59]:
#ORDENA DA ROTA MAIS UTILIZADA PARA MENOS UTILIZADA
update_data_fluxo = dados_fluxo_viagens.sort_values('contagem_viagens', ascending=False)

In [110]:
update_data_fluxo.shape

(1902, 4)

In [60]:
update_data_fluxo['contagem_viagens'][:20]

Estação de origem    Estação de Destino 
Brasil 21            Brasil 21              1296
ICC Sul              Centro Olímpico         640
Instituto de Artes   Centro Olímpico         623
Centro Olímpico      ICC Sul                 587
Deck Sul             Deck Sul                475
Centro Olímpico      Instituto de Artes      458
Ministério da Saude  Rodoviária 2            387
EQN 408/409          Instituto de Artes      378
Instituto de Artes   EQN 408/409             369
CLN 406              Instituto de Artes      320
EQN 410/411          EQN 410/411             320
Instituto de Artes   CLN 406                 277
                     EQN 410/411             276
EQN 410/411          Instituto de Artes      275
Rodoviária 2         Ministério da Saude     270
CLN 406              EQN 408/409             269
EQN 408/409          CLN 406                 266
CLN 406              EQN 410/411             264
EQN 410/411          CLN 406                 264
Rodoviária           Ministé

In [114]:
update_data_fluxo['contagem_viagens'].tail(20)

Estação de origem      Estação de Destino                            
EQS 204/205            PAT                                               1
SRTVS                  CNMP - Conselho Nacional do Ministério Público    1
EQN 410/411            TSE                                               1
EQS 204/205            EQN 408/409                                       1
STF                    EQN 408/409                                       1
EQS 104/304            Centro de Convenções                              1
                       EQN 408/409                                       1
                       ICC Sul                                           1
                       Liberty Mall                                      1
                       TSE                                               1
EQS 202 / 203          Memorial JK                                       1
                       PAT                                               1
102 Sul                EQN 408

In [61]:
update_data_fluxo['contagem_acumulada'] = update_data_fluxo['contagem_viagens'].cumsum()

In [62]:
#CRIA UMA COLUNA PARA ORIENTAR A ANÁLISE DE PARETO
update_data_fluxo['rank'] = range(1, update_data_fluxo.shape[0]+1)

In [63]:
update_data_fluxo.head()

,,contagem_viagens,contagem_acumulada,rank
Estação de origem,Estação de Destino,,,
Brasil 21,Brasil 21,1296,1296,1
ICC Sul,Centro Olímpico,640,1936,2
Instituto de Artes,Centro Olímpico,623,2559,3
Centro Olímpico,ICC Sul,587,3146,4
Deck Sul,Deck Sul,475,3621,5


In [64]:
update_data_fluxo.tail()

contagem_viagens  \
Estação de origem     Estação de Destino                       
EQS 204/205           CLN 406                              1   
                      Centro de Convenções                 1   
Praça do Povo         CLN 204                              1   
SRTVS                 Centro Olímpico                      1   
Ministério da Cultura Praça do Povo                        1   

                                            contagem_acumulada  rank  
Estação de origem     Estação de Destino                              
EQS 204/205           CLN 406                            46317  1898  
                      Centro de Convenções               46318  1899  
Praça do Povo         CLN 204                            46319  1900  
SRTVS                 Centro Olímpico                    46320  1901  
Ministério da Cultura Praça do Povo                      46321  1902

In [65]:
update_data_fluxo['freq_acumulada'] = update_data_fluxo['contagem_acumulada']/update_data_fluxo['contagem_viagens'].sum()*100

In [66]:
update_data_fluxo.head()

,,contagem_viagens,contagem_acumulada,rank,freq_acumulada
Estação de origem,Estação de Destino,,,,
Brasil 21,Brasil 21,1296,1296,1,2.797867
ICC Sul,Centro Olímpico,640,1936,2,4.179530
Instituto de Artes,Centro Olímpico,623,2559,3,5.524492
Centro Olímpico,ICC Sul,587,3146,4,6.791736
Deck Sul,Deck Sul,475,3621,5,7.817189


In [67]:
update_data_fluxo.tail()

contagem_viagens  \
Estação de origem     Estação de Destino                       
EQS 204/205           CLN 406                              1   
                      Centro de Convenções                 1   
Praça do Povo         CLN 204                              1   
SRTVS                 Centro Olímpico                      1   
Ministério da Cultura Praça do Povo                        1   

                                            contagem_acumulada  rank  \
Estação de origem     Estação de Destino                               
EQS 204/205           CLN 406                            46317  1898   
                      Centro de Convenções               46318  1899   
Praça do Povo         CLN 204                            46319  1900   
SRTVS                 Centro Olímpico                    46320  1901   
Ministério da Cultura Praça do Povo                      46321  1902   

                                            freq_acumulada  
Estação de origem     Estação de Destino                    
EQS 204/205           CLN 406                    99.991365  
                      Centro de Convenções       99.993523  
Praça do Povo         CLN 204                    99.995682  
SRTVS                 Centro Olímpico            99.997841  
Ministério da Cultura Praça do Povo             100.000000

In [68]:
#o número total de rotas que correspondem por 80% de todas as rotas
update_data_fluxo[update_data_fluxo['freq_acumulada'] <= 80].shape

(482, 4)

In [69]:
#o percentual de rotas que correspodem por 80% de todas as rotas
update_data_fluxo[update_data_fluxo['freq_acumulada'] <= 80].shape[0]/update_data_fluxo.shape[0]*100

25.341745531019978

In [70]:
update_data_fluxo.shape

(1902, 4)

In [71]:
pareto = update_data_fluxo[update_data_fluxo['freq_acumulada'] <= 80]

In [72]:
pareto = pd.DataFrame(pareto)

In [73]:
pareto.head()

,,contagem_viagens,contagem_acumulada,rank,freq_acumulada
Estação de origem,Estação de Destino,,,,
Brasil 21,Brasil 21,1296,1296,1,2.797867
ICC Sul,Centro Olímpico,640,1936,2,4.179530
Instituto de Artes,Centro Olímpico,623,2559,3,5.524492
Centro Olímpico,ICC Sul,587,3146,4,6.791736
Deck Sul,Deck Sul,475,3621,5,7.817189


In [74]:
pareto.shape

(482, 4)

In [75]:
#exporta o arquivo com as principais viagens
pareto.to_csv("../data/interm/regra_pareto_selecionado.csv", sep=',', encoding='latin-1')

In [76]:
curva_pareto = update_data_fluxo[['rank', 'freq_acumulada']]

In [77]:
#exporta dados para plotar curva de pareto
curva_pareto.to_csv("../data/interm/plotar_curva_pareto.csv", sep=',', encoding='latin-1', index=False)

In [78]:
#exporta dado completo com as infos da regra de pareto
update_data_fluxo.to_csv("../data/interm/dados_completo_curva_pareto.csv", sep=',', encoding='latin-1', index=False)

# ANÁLISE DO TEMPO DE VIAGEM

In [79]:
bikes.head()

,Gênero do usuário,data nascimento,Cidade / Região Administrativa onde habita,Idade,Data,Horário de retirada,Horário de devolução,tempo de viagem,Estação de origem,% de retiradas/estação,Estação de Destino,% de devoluções/estação,faixa_etaria
0,M,29/03/1980 00:00:00,NaN,38,2018-08-01,05:34:25,05:42:49,504.0,Rodoviária 2,0.044207,CNMP - Conselho Nacional do Ministério Público,0.008516,"(36.0, 54.0]"
1,M,1992-06-07 00:00:00,NaN,26,2018-08-01,05:39:51,05:44:13,262.0,Rodoviária,0.047566,Ministério dos Transportes,0.015021,"(18.0, 36.0]"
2,M,17/02/1975 00:00:00,NaN,43,2018-08-01,05:48:09,05:56:35,506.0,Rodoviária 2,0.044207,STF,0.016733,"(36.0, 54.0]"
3,M,1994-08-07 00:00:00,NaN,23,2018-08-01,05:49:58,06:12:44,1366.0,Rodoviária 2,0.044207,Ministério da Defesa,0.011319,"(18.0, 36.0]"
4,F,29/08/1973 00:00:00,NaN,44,2018-08-01,05:57:37,06:16:22,1125.0,Rodoviária,0.047566,Biblioteca Central,0.018444,"(36.0, 54.0]"


In [80]:
bikes.set_index('Data', inplace=True)

In [81]:
print(f"O tempo médio de percurso é de {bikes['tempo de viagem'].mean()/60} minutos.")

O tempo médio de percurso é de 18.21747443564114 minutos.


In [82]:
tempo_viagem_estacao_origem = pd.DataFrame(bikes.groupby('Estação de origem')['tempo de viagem'].resample('W').mean())

In [83]:
tempo_viagem_estacao_origem['diff_media_sec'] = tempo_viagem_estacao_origem['tempo de viagem']-bikes['tempo de viagem'].mean()

In [84]:
tempo_viagem_estacao_origem['diff_media_min'] = round(tempo_viagem_estacao_origem['diff_media_sec']/60,2)

In [85]:
tempo_viagem_estacao_origem.describe()

,tempo de viagem,diff_media_sec,diff_media_min
count,237.000000,237.000000,237.000000
mean,1113.627418,20.578951,0.343165
std,254.800624,254.800624,4.246386
min,686.862069,-406.186397,-6.770000
25%,956.634731,-136.413736,-2.270000
50%,1053.287625,-39.760841,-0.660000
75%,1204.357692,111.309226,1.860000
max,2193.031746,1099.983280,18.330000


In [86]:
#exporta dados para plotar curva de pareto
tempo_viagem_estacao_origem.to_csv("../data/interm/analise_tempo_medio.csv", sep=',', encoding='latin-1', index=True)

# TEMPO DE PERCURSO X FAIXA DE IDADE

In [87]:
bikes['faixa_etaria'].unique()

[(36.0, 54.0], (18.0, 36.0], (54.0, 72.0], (0.0, 18.0], NaN, (72.0, 90.0]]
Categories (5, interval[float64]): [(0.0, 18.0] < (18.0, 36.0] < (36.0, 54.0] < (54.0, 72.0] < (72.0, 90.0]]

In [88]:
faixa_2_idade = bikes[(bikes['Idade']  >54) & (bikes['Idade']  <= 72) ]

In [89]:
faixa_1_idade = bikes[(bikes['Idade']  > 18 ) & (bikes['Idade']  <= 36) ]

In [90]:
media_tempo_viagem_18_36 = pd.DataFrame(faixa_1_idade.groupby(['Estação de origem'])['tempo de viagem'].mean())

In [91]:
media_tempo_viagem_18_36.columns = ['media_viagem_13_36_anos']

In [92]:
media_tempo_viagem_54_72 = pd.DataFrame(faixa_2_idade.groupby(['Estação de origem'])['tempo de viagem'].mean())

In [93]:
media_tempo_viagem_54_72.columns = ['media_viagem_54_72_anos']

In [94]:
media_tempo_viagem_54_72.head()

,media_viagem_54_72_anos
Estação de origem,
102 Sul,1092.500000
Banco Central,841.000000
Biblioteca Central,934.000000
Brasil 21,1979.800000
Brasília Shopping,1192.055556


In [95]:
media_tempo_viagem_18_36.head()

,media_viagem_13_36_anos
Estação de origem,
102 Sul,1073.009843
Banco Central,994.331579
Biblioteca Central,1082.811765
Brasil 21,1818.118396
Brasília Shopping,1063.005474


# merge dataframes

In [96]:
merge_diff_idade = media_tempo_viagem_18_36.merge(media_tempo_viagem_54_72, how='inner', left_index=True, right_index=True)

In [97]:
merge_diff_idade['diff_medias_idades_seconds'] = merge_diff_idade['media_viagem_54_72_anos']-merge_diff_idade['media_viagem_13_36_anos']

In [98]:
merge_diff_idade.sort_values('diff_medias_idades_seconds', ascending=False, inplace=True)

In [99]:
top3_diff = merge_diff_idade.head(3)

In [100]:
last3_diff = merge_diff_idade.tail(3)

In [101]:
extremes_mean_diff = pd.concat([top3_diff, last3_diff])

In [102]:
extremes_mean_diff['diff_medias_idades_minutes'] = round(extremes_mean_diff['diff_medias_idades_seconds']/60,2)

In [103]:
extremes_mean_diff

,media_viagem_13_36_anos,media_viagem_54_72_anos,diff_medias_idades_seconds,diff_medias_idades_minutes
Estação de origem,,,,
Praça Buriti,1125.227378,1938.363636,813.136258,13.55
Memorial JK,1388.977929,2181.888889,792.910960,13.22
Deck Sul,1914.821839,2600.666667,685.844828,11.43
Deck Sul Kids,1923.870504,1545.000000,-378.870504,-6.31
Centro de Convenções,1273.829060,845.222222,-428.606838,-7.14
Ministério do Planejamento,767.448560,328.000000,-439.448560,-7.32


In [104]:
extremes_mean_diff.reset_index(inplace=True)

In [105]:
extremes_mean_diff = extremes_mean_diff[['Estação de origem', 'diff_medias_idades_minutes']]

In [106]:
extremes_mean_diff.columns = ['origem', 'diferença']

In [108]:
extremes_mean_diff.to_csv("../data/interm/tempo_percurso_diff_idades.csv", sep=',', encoding='latin-1', index=False)